# Testing everything

In [1]:
import pandas as pd
from datetime import datetime
from googlesearch import search
import time

In [2]:
def get_symbol_info(symbol: str):
    url1 = 'https://markets.ft.com/data/funds/tearsheet/performance?s={}'.format(symbol)
    try:
        df1 = pd.read_html(url1)[0]

        ds_perform = df1.loc[0].drop(['Unnamed: 0']).apply(lambda x: None if x[:-1] == '-' else float(x[:-1])/100.0)
        ds_perform['ft category'] = df1.loc[1]['Unnamed: 0']


        url2 = 'https://markets.ft.com/data/funds/tearsheet/historical?s={}'.format(symbol)
        df_quotes = pd.read_html(url2)[0]
        df_quotes['Date'] = df_quotes['Date'].apply(lambda x: x[:-17]).apply(lambda x: pd.to_datetime(x, infer_datetime_format = True).date())
        ds_quotes = df_quotes.set_index('Date',drop=True)['Close']

        ds_cat_perform = df1.loc[1].drop(['Unnamed: 0']).apply(lambda x: None if x[:-1] == '-' else float(x[:-1])/100.0)
        ds_cat_perform['ft category'] = df1.loc[1]['Unnamed: 0']

        url3 = 'https://markets.ft.com/data/funds/tearsheet/risk?s={}'.format(symbol)
        data = pd.read_html(url3)
        ds_risk = pd.concat([data[0], data[1]]).set_index('Unnamed: 0')['Fund']
        ds_cat_risk = pd.concat([data[0], data[1]]).set_index('Unnamed: 0')['Category average']
    except Exception as e:
        ds_perform, ds_cat_perform, ds_quotes,ds_risk, ds_cat_risk = None, None, None, None, None
    
    return ds_perform, ds_cat_perform, ds_quotes,ds_risk, ds_cat_risk
   

In [3]:
fund_list_file = 'fund_symbols.csv'
df = pd.read_csv(fund_list_file)[['symbol','name','category','risk','currency']]
data = []
data_cat = []
df_quote = pd.DataFrame()
for _,fund in df.iterrows():
    ds_perform, ds_cat_perform, ds_quotes,ds_risk, ds_cat_risk = get_symbol_info(fund['symbol'])
    if ds_perform is not None:
        lst = fund.to_list()
        lst += ds_perform.to_list()
        lst += [float(ds_risk['Standard deviation'][:-1])/100.0]
        lst += [ds_quotes.std()]
        lst_cat = fund[['category','risk']].to_list()
        lst_cat += ds_cat_perform.to_list()
        data.append(lst)
        data_cat.append(lst_cat)
        df_quote[fund['symbol']] = ds_quotes
columns = ['symbol','name','category','risk','currency','5 years', '3 years', '1 year', '6 months', '3 months', '1 month', 'ft category', '1 year std', '1 month std']
columns_cat = ['m category','risk', '5 years', '3 years', '1 year', '6 months', '3 months', '1 month', 'ft category']
df_fund = pd.DataFrame(data=data, columns=columns).drop_duplicates(subset=['symbol'])
df_cat = pd.DataFrame(data=data_cat, columns=columns_cat).drop_duplicates(subset=['ft category'])

In [4]:
df_cat.sort_values(by='1 month',ascending=False)

,m category,risk,5 years,3 years,1 year,6 months,3 months,1 month,ft category
41,Ações Sector Saúde,6,0.0818,0.0896,-0.0366,0.0570,0.0212,0.1097,Sector Equity Healthcare
63,Ações EUA Cap. Grande Valor,6,0.0879,0.0944,0.1043,0.0456,-0.0259,0.0654,US Large-Cap Value Equity
36,Obrigações Ásia,4,0.0413,0.0320,0.0218,0.0195,0.0162,0.0470,USD Moderate Allocation
45,Ações Sector Saúde,6,0.0621,0.0631,0.0429,-0.0010,-0.0212,0.0450,Global Equity Income
16,Obrigações Globais USD Hedged,3,0.0227,0.0032,0.0475,0.0568,0.0684,0.0423,Global Bond - USD Hedged
40,Ações Sector Infra-estrutura,6,0.0434,0.0419,0.0930,0.0627,-0.0244,0.0378,Sector Equity Infrastructure
48,Obrigações Globais Inflation Linked - USD Hedged,4,0.0396,0.0290,0.0821,0.0650,0.0362,0.0371,Global Inflation-Linked Bond - USD Hedged
75,Obrigações Flexíveis USD,4,0.0263,0.0116,0.0444,0.0493,0.0535,0.0322,USD Flexible Bond
15,Obrigações Curto Prazo Diversificado USD,2,0.0241,0.0165,0.1079,0.1049,0.0826,0.0302,USD Diversified Bond - Short Term
7,Alternativo Market Neutral - Ações USD,4,0.0445,0.0419,0.1757,0.1457,0.0922,0.0299,Equity Market Neutral USD


In [5]:
df_fund = df_fund.drop_duplicates(subset=['symbol']).sort_values(by='1 month',ascending=False)
df_fund


,symbol,name,category,risk,currency,5 years,3 years,1 year,6 months,3 months,1 month,ft category,1 year std,1 month std
41,LU0122380701:USD,BGF World Healthscience E2,Ações Sector Saúde,6,USD,0.1078,0.1086,0.0858,0.1118,0.0255,0.0734,Sector Equity Healthcare,0.1291,0.479861
64,LU1769942662:USD,DWS Invest CROCI US Dividends USD LC,Ações EUA Cap. Grande Valor,6,USD,0.1320,0.1446,0.2006,0.1176,0.0008,0.0700,US Large-Cap Value Equity,0.0847,3.890566
63,LU1769942829:USD,DWS Invest CROCI US Dividends USD LD,Ações EUA Cap. Grande Valor,6,USD,0.1260,0.1384,0.1942,0.1146,-0.0006,0.0696,US Large-Cap Value Equity,0.0846,3.745453
40,LU0384381660:USD,MS INVF Global Infrastructure A,Ações Sector Infra-estrutura,6,USD,0.0569,0.0469,0.1185,0.1121,0.0055,0.0674,Sector Equity Infrastructure,0.1214,0.710609
57,LU0329760853:EUR,DWS Invest Global Infras NC,Ações Sector Infra-estrutura,6,EUR,0.0624,0.0588,0.1413,0.1126,-0.0244,0.0596,Sector Equity Infrastructure,0.1152,2.751332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LU0605512192:EUR,Fidelity US Dollar Bond A-Dis-USD,Obrigações Diversificado USD,3,USD,-0.0269,-0.0506,-0.1628,-0.1127,-0.0354,-0.0347,Other Bond,0.0604,0.091076
25,LU1071462615:EUR,Pictet TR - Agora P EUR,Alternativo Market Neutral - Ações EUR,4,EUR,-0.0022,-0.0165,-0.0387,-0.0341,0.0212,-0.0355,Equity Market Neutral EUR,0.0869,1.253790
24,LU1229532970:EUR,Pictet TR - Agora P dy EUR,Alternativo Market Neutral - Ações EUR,4,EUR,-0.0021,-0.0165,-0.0386,-0.0341,0.0211,-0.0356,Equity Market Neutral EUR,0.0869,1.254396
14,LU1245572752:EUR,T. Rowe Price Dyn Glb Bd Ah EUR,Obrigações Flexível Globais-EUR Hedged,4,EUR,-0.0109,-0.0013,0.0040,0.0395,-0.0031,-0.0433,Global Flexible Bond - EUR Hedged,0.0537,0.064302


In [6]:
df_fund['1 month racio'] = df_fund['1 month'] / df_fund['1 month std']
df_fund.sort_values(by='1 month racio',ascending=False)


,symbol,name,category,risk,currency,5 years,3 years,1 year,6 months,3 months,1 month,ft category,1 year std,1 month std,1 month racio
51,IE00B2R34X65:USD,PIMCO GIS Glb Bd Ex-US E USD Inc,Obrigações Globais USD Hedged,3,USD,0.0230,-0.0041,0.0345,0.0434,0.0562,0.0443,Global Bond - USD Hedged,0.0709,0.049340,0.897845
75,IE00B8HX1V50:USD,Neuberger Berman Strat Inc A USD Acc,Obrigações Flexíveis USD,4,USD,0.0360,0.0269,0.0467,0.0414,0.0557,0.0474,USD Flexible Bond,0.0625,0.098307,0.482164
34,LU0712123271:USD,MS INVF Global Fixed Income Opps A,Obrigações Flexível Globais,3,USD,0.0353,0.0106,0.0608,0.0539,0.0374,0.0265,Global Flexible Bond,0.0682,0.087842,0.301679
5,IE00B6SCG018:EUR,T. Rowe Price Dyn Glb Inv Gr Bd A USD,Obrigações Flexível Globais,4,USD,0.0059,-0.0003,-0.0842,-0.0553,-0.0307,0.0347,EUR Flexible Allocation - Global,0.0730,0.135618,0.255865
68,LU1512665842:EUR,GS Global Infras Eq Oth Ccy Acc EUR,Ações Sector Infra-estrutura,5,EUR,0.0598,0.0479,0.1231,0.1046,-0.0330,0.0511,Sector Equity Infrastructure,0.1131,0.205083,0.249168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LU0605512192:EUR,Fidelity US Dollar Bond A-Dis-USD,Obrigações Diversificado USD,3,USD,-0.0269,-0.0506,-0.1628,-0.1127,-0.0354,-0.0347,Other Bond,0.0604,0.091076,-0.381002
4,LU0900496406:EUR,JPM China D (acc) EUR,Ações China,6,EUR,-0.0416,-0.0782,-0.2274,-0.1696,-0.0833,-0.0528,Global Emerging Markets Corporate Bond - EUR B...,0.0658,0.116719,-0.452368
30,LU1460782227:EUR,HSBC GIF Global Bond Total Ret AC,Obrigações Flexível Globais,3,USD,-0.0018,-0.0210,-0.0463,0.0124,-0.0084,-0.0304,Multistrategy EUR,0.0472,0.052717,-0.576664
14,LU1245572752:EUR,T. Rowe Price Dyn Glb Bd Ah EUR,Obrigações Flexível Globais-EUR Hedged,4,EUR,-0.0109,-0.0013,0.0040,0.0395,-0.0031,-0.0433,Global Flexible Bond - EUR Hedged,0.0537,0.064302,-0.673384


In [7]:
df_quote = df_quote.dropna()

In [8]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

mu = mean_historical_return(df_quote)
S = CovarianceShrinkage(df_quote).ledoit_wolf()

In [9]:
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
cleaned_weights = [(s,w) for s,w in cleaned_weights.items() if w > 0.0]
cleaned_weights

[('LU1216622305:USD', 0.04857),
 ('LU0900495697:USD', 0.09578),
 ('LU0900496406:EUR', 0.12849),
 ('IE00B6116G19:USD', 0.12107),
 ('LU0837970705:USD', 0.03219),
 ('IE00BKJ9MZ69:USD', 0.06932),
 ('LU1245572752:EUR', 0.05774),
 ('IE00BDRVT430:USD', 0.01916),
 ('LU0468289250:EUR', 0.00932),
 ('LU1229532970:EUR', 0.00264),
 ('LU1071462615:EUR', 0.00223),
 ('LU1062842700:EUR', 0.08638),
 ('LU0546916452:USD', 0.05113),
 ('LU0055114457:USD', 0.00768),
 ('LU0712123271:USD', 0.02041),
 ('LU0792749094:EUR', 0.00565),
 ('LU0076315455:EUR', 0.04043),
 ('LU1176912761:EUR', 0.01848),
 ('LU0455550201:USD', 0.02603),
 ('IE00B2R34X65:USD', 0.03394),
 ('LU0532656005:USD', 0.04297),
 ('LU0605512192:EUR', 0.08038)]

In [16]:
df_port = df_fund[df_fund['symbol'].isin([s for s,_ in cleaned_weights])]
df_port

,symbol,name,category,risk,currency,5 years,3 years,1 year,6 months,3 months,1 month,ft category,1 year std,1 month std,1 month racio
51,IE00B2R34X65:USD,PIMCO GIS Glb Bd Ex-US E USD Inc,Obrigações Globais USD Hedged,3,USD,0.0230,-0.0041,0.0345,0.0434,0.0562,0.0443,Global Bond - USD Hedged,0.0709,0.049340,0.897845
9,IE00B6116G19:USD,Man GLG Alpha Select Alt DLH USD,Alternativo Market Neutral - Ações USD,5,USD,0.0925,0.0839,0.2486,0.2040,0.0912,0.0280,Equity Market Neutral USD,0.0873,0.903470,0.030992
19,IE00BDRVT430:USD,Man GLG Innovation Eq Alt DN USD Acc,Ações Long/Short Global,4,USD,0.0419,0.0436,0.1290,0.1266,0.0738,0.0272,Equity Market Neutral USD,0.1091,0.300174,0.090614
34,LU0712123271:USD,MS INVF Global Fixed Income Opps A,Obrigações Flexível Globais,3,USD,0.0353,0.0106,0.0608,0.0539,0.0374,0.0265,Global Flexible Bond,0.0682,0.087842,0.301679
48,LU0455550201:USD,UBS(Lux) BS Glb Infl Lnkd$ P-acc,Obrigações Globais Inflation Linked - USD Hedged,4,USD,0.0456,0.0313,0.0791,0.0666,0.0460,0.0257,Global Inflation-Linked Bond - USD Hedged,0.0637,0.428268,0.060009
43,LU0076315455:EUR,Nordea 1 - European Covered Bd Op E EUR,Obrigações Corp. EUR,3,EUR,-0.0117,-0.0411,-0.0980,-0.0543,-0.0019,0.0155,EUR Corporate Bond,0.0436,0.088345,0.175449
10,LU0837970705:USD,abrdn China A Share Sus Eq A Acc USD,Ações China - A Shares,6,USD,0.0383,0.0007,-0.2235,-0.1100,0.1269,0.0088,China Equity,0.1828,0.389628,0.022586
32,LU0546916452:USD,NN (L) EmMkts Dbt (LclCcy) X Cap USD,Obrigações Emergentes Moeda Local,4,USD,-0.0143,-0.0289,0.0258,0.0360,0.0225,0.0022,Global Emerging Markets Bond - Local Currency,0.0582,0.457368,0.004810
11,IE00BKJ9MZ69:USD,Neuberger Berman China Eq USD A Acc,Ações China,6,USD,0.0590,0.0353,-0.1617,-0.0751,0.1620,0.0010,China Equity,0.1454,0.532071,0.001879
23,LU0468289250:EUR,BSF Emerging Mkts Shrt Dur Bd E2 EUR,Obrigações Emergentes Globais,5,EUR,-0.0183,-0.0311,-0.0549,-0.0184,0.0083,0.0000,EUR Diversified Bond - Short Term,0.0360,0.053510,0.000000
